In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
! pip install -U pandas_datareader

  Found existing installation: pandas-datareader 0.7.0
    Uninstalling pandas-datareader-0.7.0:
      Successfully uninstalled pandas-datareader-0.7.0


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [36]:
import pandas as pd
import mplfinance as mpf
import numpy as np
import pandas_datareader.data as web
import talib
import datetime

In [37]:
data = web.DataReader("600797.SS", "yahoo", datetime.datetime(2018,1,1), datetime.datetime(2019,1,1))
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-01-02,11.93,11.75,11.83,11.87,9507327.0,11.733099
2018-01-03,12.26,11.81,11.81,12.17,16068391.0,12.029640
2018-01-04,12.18,12.01,12.06,12.09,10067364.0,11.950562
2018-01-05,12.38,12.03,12.16,12.04,12695730.0,11.901140
2018-01-08,12.55,11.70,11.96,12.32,21512391.0,12.177910


### MACD
MACD称为异同移动平均线，属于趋势类指标，由快速线DIF、慢速线DEA组成。在股票行情显示界面中，DIF和DEA是以零轴为中心轴，上下交叉移动的两条线，红绿柱状图BAR为辅助显示的指标。MACD计算方法如下所示：

DIF（差离值） = EMA（12日收盘价移动平均线）- EMA（26日收盘价移动平均线）

DEA（差离平均值） = DIF的9日移动平均线

BAR（柱状值） = 2 ×（DIF-DEA）

MACD本质上以短期移动平均线与长期移动平均线之间的聚合与分离状况（±DIF值）来对行情趋势判断，DEA则是对DIF值进行N日移动平滑用于辅助观察变化。理论上，在持续加速的涨势中，短期EMA线在长期EMA线之上，其间的正离差值(+DIF)会越来越大，DIF距离DEA也会变大，BAR柱状线也会扩展；反之，在加速跌势中负离差值(-DIF)会越来越小。当行情即将反转时，DIF与DEA距离会逐渐减小，也就是指标出现背离的情形，对应于股价可体现为第二轮上涨/下跌虽然超越了第一轮股价的高点/低点，但第二轮运行的动能比第一轮降低了，预示着趋势已经是强弩之末的态势了。

### KDJ
KDJ是最为常用的震荡类指标。在介绍KDJ指标之前，先介绍下拉里·威廉斯（Larry Williams）提出的威廉指标（Williams %R，简称W%R）。威廉指标的计算首先选定一个特定的时间跨度（比如14日），然后找出这一特定区间的最高价和最低价，构成一个价格变化区间，而后分析这一时间跨度最后一个时间点的收盘价与期间最高价和最低价的相对位置，以此来衡量市场的超买或超卖现象。

KDJ指标的最早雏形是由芝加哥期货交易商George Lane提出的KD指标，该指标又被称为随机震荡指标（Stochastic Oscillator Indicator）。从最基础的交易思想上看，KD指标的分析思想与威廉指标类似，均使用特定时间跨度中的最后收盘价与该时间跨度内的最高价和最低价的相对位置来推测市场的超买和超卖情况。与威廉指标不同的是，随机震荡指标在收盘价与最高价和最低价的相对位置的比值上，又融合了移动平均的思想，用更多的信息来捕捉市场的超买、超卖现象。KDJ指标则是在随机震荡指标K线和D线的基础上增加了一条J线，进一步提高了随机震荡指标对市场买卖信号捕捉的准确度。

In [61]:
# macd
macd_dif, macd_dea, macd_bar  = talib.MACD(data.Close.values, fastperiod=12, slowperiod=26, signalperiod=9)

# kdj
k,d = talib.STOCH(data.High.values, data.Low.values, data.Close.values,\
                                       fastk_period=9, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

j = 3 * k - 2 * d

addplot = [
    # macd
    mpf.make_addplot(macd_bar,type='bar', panel=2),
    mpf.make_addplot(macd_dea, panel=2),
    mpf.make_addplot(macd_dif, panel=2),
    mpf.make_addplot(k, panel=3),
    mpf.make_addplot(d, panel=3,),
    mpf.make_addplot(j, panel=3),
    
    
]


mc = mpf.make_marketcolors(up='red', down='g',volume={'up': 'red', 'down': 'green'})
s = mpf.make_mpf_style(marketcolors=mc)
fig, axe = mpf.plot(data, title="600797 day_k",mav=(20,30,60),type='candle',volume=True, returnfig=True,figscale=1.5, style=s, datetime_format='%Y-%m-%d',
                   tight_layout=True, addplot=addplot)
axe[0].legend(["M20", "M30", "M60"])
axe[5].legend(["M20", "M30"])



KeyError: 'Unrecognized kwarg="style"'